In [1]:
import pyscf
import pyscf.qmmm
from pyscf import gto, scf
import numpy as np
import matplotlib.pyplot as plt
from pyscf.geomopt.berny_solver import optimize
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
from pyscf import lib
import inspect
from functools import reduce
from pyscf.scf import cphf
angstrom = 1 / 0.52917721067
from pyscf.scf._response_functions import _gen_rhf_response 

In [2]:
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV
def fc(calc,deltaZ):
    mf = pyscf.qmmm.mm_charge(calc, calc.mol.atom_coords(), deltaZ)  # now is add_mm_charge
    class NoSelfQMMM(mf.__class__):
        def energy_nuc(self):
            q = self.mol.atom_charges().astype(np.float).copy()
            q1 =q+ np.asarray(deltaZ) 
            return self.mol.energy_nuc(q1)
    return(NoSelfQMMM(mf,mf.mm_mol))

In [65]:
mol = gto.M(atom='H 0 0 0; H 0 0 1.2', unit="Bohr",basis="3-21g")
mf = scf.RHF(mol)
e=mf.scf()

converged SCF energy = -1.11377437897079


In [66]:
C=mf.mo_coeff
P=mf.make_rdm1()
O=np.diag(mf.mo_occ)
mo_coeff=mf.mo_coeff
mo_occ=mf.mo_occ
mo_energy=mf.mo_energy

In [67]:
mfa=fc(mf,[.01,-.01])
mfa.run()
P1=(mfa.make_rdm1()-P)
#C1=(mfa.mo_coeff*[1,-1]-C)
C1=mfa.mo_coeff-C

converged SCF energy = -1.11394322077261


In [68]:
hs=mf.Hessian()

In [69]:
#fx is a function  fx(dm_mo) => v1_mo.  A function to generate the induced potential.
#also passes through  pyscf.scf._response_functions._gen_rhf_response
fxh=pyscf.hessian.rhf.gen_vind(mf,mf.mo_coeff,mf.mo_occ)
#h1vo come from h1ao generated by the function: is a list with an element per each atom
h1ao=hs.make_h1(mf.mo_coeff,mf.mo_occ)
#s1ao comes from the espextation value of the nabla operator
s1ao=mol.intor('int1e_ipovlp', comp=3).shape        

In [70]:
from pyscf.prop.polarizability.rhf import Polarizability
plo=Polarizability(mf)
fxp=plo.gen_vind(mf,mf.mo_coeff,mf.mo_occ)

In [71]:
nao, nmo = mo_coeff.shape
mocc = mo_coeff[:,mo_occ>0] 
nocc = mocc.shape[1]
vresp = _gen_rhf_response(mf, mo_coeff, mo_occ, hermi=1)

In [72]:
mf.mo_coeff,mf.mo_occ

(array([[ 0.30635213,  0.09453639,  0.73707525, -1.17123919],
        [ 0.28640484,  1.90830225, -0.65588697,  1.36009828],
        [ 0.30635213, -0.09453639,  0.73707525,  1.17123919],
        [ 0.28640484, -1.90830225, -0.65588697, -1.36009828]]),
 array([2., 0., 0., 0.]))

In [73]:
nocc

1

In [74]:
mo_occ

array([2., 0., 0., 0.])

In [99]:
h1mo=DeltaV(mol,[.01,-.01])
h1vo=(C@h1mo@C.T)
#h1vo = np.vstack(h1vo)
h1vo

array([[ 0.0018274 , -0.00143456,  0.00267152, -0.00367788],
       [-0.00143456, -0.01280172, -0.0030272 ,  0.00501011],
       [ 0.00267152, -0.0030272 ,  0.0091452 , -0.0125959 ],
       [-0.00367788,  0.00501011, -0.0125959 ,  0.01542078]])

In [76]:
import numpy
from pyscf import lib

def solve_nos1(fvind, mo_energy, mo_occ, h1,max_cycle=20, tol=1e-9, hermi=False):
    '''For field independent basis. First order overlap matrix is zero'''
    e_a = mo_energy[mo_occ==0]
    e_i = mo_energy[mo_occ>0]
    e_ai = 1 / lib.direct_sum('a-i->ai', e_a, e_i)
    mo1base = h1 * -e_ai

    def vind_vo(mo1):
        v = fvind(mo1.reshape(h1.shape)).reshape(h1.shape)
        v *= e_ai
        return v.ravel()
    mo1 = lib.krylov(vind_vo, mo1base.ravel(),tol=tol, max_cycle=max_cycle, hermi=hermi)
    return mo1.reshape(h1.shape), None


In [77]:
mo_energy, mf.mo_occ

(array([-0.62711131,  0.29432989,  0.86597663,  1.67350706]),
 array([2., 0., 0., 0.]))

In [106]:
dC=solve_nos1(fxh, mf.mo_energy ,mf.mo_occ,h1vo[nocc:,:])[0]
print(dC)

[[-5.41921591e-05  1.75594351e-02  4.28794138e-03 -5.38370247e-03]
 [-2.78769656e-05  3.12965332e-03 -7.12439230e-03  1.06503147e-02]
 [ 4.10035943e-04 -2.74355091e-03  5.95609050e-03 -7.76912065e-03]]


In [108]:
s1 = numpy.zeros_like(h1vo)
dC=cphf.solve(fxp, mo_energy ,mf.mo_occ,h1vo,s1)[0]
dC

ValueError: cannot reshape array of size 4 into shape (1,1)

In [87]:
mf.mo_coeff

array([[ 0.30635213,  0.09453639,  0.73707525, -1.17123919],
       [ 0.28640484,  1.90830225, -0.65588697,  1.36009828],
       [ 0.30635213, -0.09453639,  0.73707525,  1.17123919],
       [ 0.28640484, -1.90830225, -0.65588697, -1.36009828]])

In [97]:
mfa=fc(mf,[.01,-.01])
mfa.run()
mfa.mo_coeff

converged SCF energy = -1.11394322077261


array([[ 0.31111991, -0.09604144,  0.74626221,  1.16402023],
       [ 0.28920255, -1.90462351, -0.67419742, -1.3557025 ],
       [ 0.30160925,  0.09301498,  0.72780502, -1.17836745],
       [ 0.28356765,  1.91193684, -0.63754517,  1.36430324]])

In [107]:
(mfa.mo_coeff-mf.mo_coeff)[:,0]

array([ 0.00476779,  0.00279771, -0.00474288, -0.00283719])